In [ ]:
from functools import wraps
import time


def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [1]:
import pandas as pd
import json
from pprint import pformat
import sqlite3
conn = sqlite3.connect('lotr.db')

In [ ]:
@timeit
def get_set_from_json(file_name:str):
    with open(file_name) as f:
        data = json.load(f)

    lotr_set = data['data']['cards']

    return lotr_set

@timeit
def get_all_keys(json_array:list):
    all_keys = set().union(*(d.keys() for d in json_array))
    return all_keys

lotr_set = get_set_from_json('lotr.json')
all_keys = get_all_keys(lotr_set)

In [ ]:
drop_list = ['availability', 'edhrecRank', 'colorIdentity', 'foreignData', 'frameVersion', 'language', 'layout', 
    'printings', 'securityStamp', 'setCode', 'boosterTypes', 'hasAlternativeDeckLimit', 'edhrecSaltiness', 'flavorName', 
    'borderColor', 'isReprint', 'asciiName']

In [ ]:
all_keys = sorted(all_keys)
print(len(all_keys),all_keys)
for d in drop_list:
    all_keys.remove(d)
print(len(all_keys),all_keys)

In [ ]:
set_lists = {}
set_dicts = {}

for card in lotr_set:
    for key in card:
        if key not in all_keys:
            continue
        if isinstance(card[key], list):
            if key not in set_lists.keys():
                set_lists[key] = []
        if isinstance(card[key], dict):
            if key not in set_dicts.keys():
                set_dicts[key] = []

for i, card in enumerate(lotr_set):
    
    for key in card:
        if not card[key]:
            card[key] = "None"
        if isinstance(card[key], list):
            if key not in set_lists.keys():
                set_lists[key] = []
            for item in card[key]:
                new_card = {}
                new_card[key] = card[key]
                new_card['name'] = card['name']
                new_card['number'] = card['number']
                new_card['uuid'] = card['uuid']
                set_lists[key].append(new_card)
        if isinstance(card[key], dict):
            if key not in set_dicts.keys():
                set_dicts[key] = []
            new_card = {}
            new_card = card[key]
            new_card['name'] = card['name']
            new_card['number'] = card['number']
            new_card['uuid'] = card['uuid']
            set_dicts[key].append(new_card)

print(set_lists.keys())
print(set_dicts.keys())


In [ ]:
lotr = {}
for key in all_keys:
    if key in set_lists or key in set_dicts:
        continue
    lotr[key] = [lotr_set[i].get(key, None) for i in range(len(lotr_set))]

In [ ]:
lotr_df = pd.DataFrame(lotr)
lotr_df.head()

In [ ]:
print(pformat(set_lists['keywords'][0]))

In [ ]:
print(pformat(set_lists))

In [ ]:
for key in set_lists:
    print(key, len(set_lists[key]))

In [ ]:
print(pformat(set_lists['finishes'][0]))

In [ ]:
print(pformat(set_dicts))

In [ ]:

for k in set_dicts:
    print(k)
    print(set_dicts[k])
    #print(set_dicts[k][0].keys())
    #df = pd.DataFrame(set_dicts[k])
    #print(df.head())
    #lotr_df = lotr_df.drop(columns=set_dicts[k][0].keys())
    # df.to_sql(f'{k}', conn, if_exists='replace', index=False)


In [ ]:
for k in set_lists:
    print(k)
    # print(pformat(set_lists[k][0]))
    # create dataframe and save it as a new table in lotr.db
    # df = pd.DataFrame(set_lists[k])
    # lotr_df = lotr_df.drop(columns=[k])
    # df.to_sql(f'{k}', conn, if_exists='replace', index=False)


In [ ]:
lotr_df.to_sql('cards', conn, if_exists='replace', index=False)

In [ ]:
print(pformat(set_lists['finishes']))

In [ ]:
# read data from faltantes.json
with open('faltantes.json') as _file:
    #faltantes = json.dumps(_file)
    faltantes = json.loads(_file.read())

In [ ]:
print(faltantes)
# for each key on json faltantes, create a new table on lotr.db


In [ ]:
for i in range(5):
    globals()[f'var{i}'] = f'Value for variable {i}'
    print(globals()[f'var{i}'])  # Prints 'Value for variable 0', 'Value for variable 1', etc.
